In [3]:
import gym
from gym.envs.robotics import utils
import numpy as np

from stable_baselines3 import  SAC, DDPG, TD3
import stable_baselines3
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.her import   HerReplayBuffer
# from stable_baselines3.td3.policies import MultiInputPolicy
from stable_baselines3.her.goal_selection_strategy import GoalSelectionStrategy
from fetch_image_env import FetchReachImageEnv
import torch as th
from torch import nn
import utils
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3 import  SAC, DDPG, TD3
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.her import   HerReplayBuffer
from stable_baselines3.her.goal_selection_strategy import GoalSelectionStrategy
from fetch_image_env import FetchReachImageEnv, make_sb3_env, make
import gym

from stable_baselines3.common.env_checker import check_env
from gym.wrappers import Monitor
from video import VideoRecorder

In [7]:

env=make_sb3_env(env_name="fetch_reach", action_repeat=2, max_episode_steps=50, seed=10, fixed=False, reward_type="dense")
# env = Monitor(env, './video1', force=True)
check_env(env)

[10]


In [8]:
env

<TimeLimit<ActionRepeat<FetchReachImageEnv instance>>>

In [3]:
check_env(env)

In [19]:
from stable_baselines3.common.env_util import make_vec_env
vec_env = make_vec_env(lambda: env, n_envs=1)

In [27]:
obs = vec_env.reset()

In [28]:
vec_env.normalize_obs(obs)

AttributeError: 'DummyVecEnv' object has no attribute 'normalize_obs'

In [29]:
vec_norm= VecNormalize(vec_env)

In [31]:
obs

OrderedDict([('achieved_goal',
              array([[1.3418326 , 0.7491004 , 0.53472275]], dtype=float32)),
             ('desired_goal',
              array([[1.426043  , 0.75990236, 0.61896676]], dtype=float32)),
             ('desired_goal_image',
              array([[[[114, 114, 114, ..., 114, 114, 114],
                       [185, 185, 220, ..., 114, 114, 114],
                       [255, 255, 255, ..., 114, 114, 114],
                       ...,
                       [ 53,  53,  59, ...,  65,  65,  65],
                       [ 59,  65,  65, ...,  65,  65,  65],
                       [ 65,  65,  65, ...,  65,  65,  65]],
              
                      [[219, 219, 219, ..., 219, 219, 219],
                       [237, 237, 246, ..., 219, 219, 219],
                       [255, 255, 255, ..., 219, 219, 219],
                       ...,
                       [ 67,  66,  66, ...,  65,  65,  65],
                       [ 65,  65,  65, ...,  65,  65,  65],
                 

In [30]:
vec_norm.normalize_obs(obs)

OrderedDict([('achieved_goal',
              array([[1.3418326 , 0.7491004 , 0.53472275]], dtype=float32)),
             ('desired_goal',
              array([[1.426043  , 0.75990236, 0.61896676]], dtype=float32)),
             ('desired_goal_image',
              array([[[[10., 10., 10., ..., 10., 10., 10.],
                       [10., 10., 10., ..., 10., 10., 10.],
                       [10., 10., 10., ..., 10., 10., 10.],
                       ...,
                       [10., 10., 10., ..., 10., 10., 10.],
                       [10., 10., 10., ..., 10., 10., 10.],
                       [10., 10., 10., ..., 10., 10., 10.]],
              
                      [[10., 10., 10., ..., 10., 10., 10.],
                       [10., 10., 10., ..., 10., 10., 10.],
                       [10., 10., 10., ..., 10., 10., 10.],
                       ...,
                       [10., 10., 10., ..., 10., 10., 10.],
                       [10., 10., 10., ..., 10., 10., 10.],
                 

In [24]:
from stable_baselines3.common.vec_env import (
    DummyVecEnv,
    VecEnv,
    VecNormalize,
    VecTransposeImage,
    is_vecenv_wrapped,
    unwrap_vec_normalize,
)

uvn = unwrap_vec_normalize(env)
# uvn.get_original_obs()

In [26]:
unwrap_vec_normalize(env)

In [7]:
from gym import spaces
isinstance(env.observation_space, spaces.Dict)

True

In [8]:
import os
os.getcwd()

'/home/sandipan/rl/research/proto_goal'

In [11]:
monitor_dir=os.path.join(os.getcwd(), "new_env")
env = Monitor(env, monitor_dir)


/home/sandipan/miniconda3/envs/pad/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:
env.observation_space.spaces["image_observation"]

Box(0, 255, (3, 84, 84), uint8)

In [6]:
env

<TimeLimit<ActionRepeat<FetchReachImageEnv instance>>>

In [13]:
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecEnv
vec_env = DummyVecEnv(env)

TypeError: 'Monitor' object is not iterable

In [ ]:
class GoalReplayBuffer:

    # check this out 
    # https://github.com/vitchyr/rlkit/blob/v0.1.2/rlkit/data_management/obs_dict_replay_buffer.py

    def __init__(self, buffer_size, sample_func, env_params):
        self.env_params= env_params
        self.T= env_params["max_env_steps"]
        self.size= buffer_size // self.T
        self.current_size = 0
        self.n_transitions_stored = 0
        self.sample_func = sample_func
        # see observation_spec for dimensions
        self.episode = defaultdict(list)
        self.buffers = {'observation': np.empty([self.size, self.T , *self.env_params['observation']], dtype=np.float32),
                'achieved_goal': np.empty([self.size, self.T, *self.env_params['achieved_goal']], dtype=np.float32),
                'achieved_goal_image':np.empty([self.size, self.T , *self.env_params['achieved_goal_image']], dtype=np.uint8),
                'achieved_goal_image_next':np.empty([self.size, self.T , *self.env_params['achieved_goal_image']], dtype=np.uint8),
                'desired_goal': np.empty([self.size, self.T, *self.env_params['desired_goal']],dtype=np.float32),
                'desired_goal_image':np.empty([self.size, self.T, *self.env_params['desired_goal_image']],dtype=np.uint8),
                'actions': np.empty([self.size, self.T, *self.env_params['actions']], dtype=np.float32)
                
                }

    def sample(self, batch_size):
        temp_buffers={}
        for key in self.buffers.keys():
            temp_buffers[key]=self.buffers[key][:self.current_size]
        # temp_buffers['achieved_goal_next'] = temp_buffers['achieved_goal'][:, 1:, :]
        # temp_buffers['achieved_goal_image_next']=temp_buffers['achieved_goal_image'][:, 1:, :]

        # sample transitions
        transitions = self.sample_func(temp_buffers, batch_size)
        
        # discounts = np.ones((batch_size, 1), dtype=np.float32) * discount
        # discounts = torch.as_tensor(discounts, device=self.device)
        return transitions

    def __len__(self):
        return self.current_size


    def add(self,obs,next_obs,action):
        self.episode['observation'].append(np.array(obs['image_observation']))
        self.episode['achieved_goal'].append(np.array(obs['achieved_goal']))
        self.episode['achieved_goal_image'].append(np.array(obs['image_observation']))

        self.episode['achieved_goal_image_next'].append(np.array(next_obs['image_observation']))
        self.episode['desired_goal'].append(np.array(obs['desired_goal']))
        self.episode['desired_goal_image'].append(np.array(obs['desired_goal_image']))
        self.episode['action'].append(np.array(action))
        
        if len(self.episode) ==  self.T:
            self.store_episode([np.array(self.episode['observation']),self.episode['achieved_goal'],
                                self.episode['achieved_goal_image_next'],self.episode['desired_goal'],
                                self.episode['desired_goal_image'],self.episode['action']])
            self.episode = defaultdict(list)

    def store_episode(self, episode_batch):
        observation,achieved_goal, achieved_goal_image, desired_goal,desired_goal_image,actions = episode_batch
        batch_size = observation.shape[0]
        # with self.lock:
        idxs = self._get_storage_idx(inc=batch_size)
        # store the informations
        self.buffers['achieved_goal'][idxs] = achieved_goal
        self.buffers['achieved_goal_image'][idxs] = achieved_goal_image
        self.buffers['achieved_goal_image_next'][idxs] = achieved_goal_image
        self.buffers['desired_goal'][idxs] = desired_goal
        self.buffers['desired_goal_image'][idxs] = desired_goal_image
        self.buffers['actions'][idxs] = actions
        self.n_transitions_stored += self.T * batch_size

    def _get_storage_idx(self, inc=None):
        inc = inc or 1
        if self.current_size+inc <= self.size:
            idx = np.arange(self.current_size, self.current_size+inc)
        elif self.current_size < self.size:
            overflow = inc - (self.size - self.current_size)
            idx_a = np.arange(self.current_size, self.size)
            idx_b = np.random.randint(0, self.current_size, overflow)
            idx = np.concatenate([idx_a, idx_b])
        else:
            idx = np.random.randint(0, self.size, inc)
        self.current_size = min(self.size, self.current_size+inc)
        if inc == 1:
            idx = idx[0]
        return idx
 

In [ ]:
class HERSampler:
    def __init__(self, replay_strategy, replay_k, reward_func=None):
        self.replay_strategy= replay_strategy
        self.replay_k= replay_k # Number of virtual transitions to create per real transition, by sampling new goals.
        # self.image_encoder= image_encoder 
        
        if self.replay_strategy == 'future':
            self.future_p= 1 - (1. / (1 + replay_k))
        else:
            self.future_p = 0

        self.reward_func = reward_func



     
    
    
    def sample_her_transitions(self, episode_batch, batch_size_in_transitions ):
        T = episode_batch['actions'].shape[1]
        rollout_batch_size= episode_batch['actions'].shape[0]
        batch_size = batch_size_in_transitions
        # print(rollout_batch_size,batch_size,T)
        # np.random.randint - low-inclusive and high-exclusive, hence max_env_steps+1 
        episode_idxs = np.random.randint(0, rollout_batch_size, batch_size)
        t_samples = np.random.randint(T, size=batch_size)
        transitions = {key: episode_batch[key][episode_idxs, t_samples].copy() for key in episode_batch.keys()}
        
          # her idx
        her_indexes = np.where(np.random.uniform(size=batch_size) < self.future_p)
        # TODO try future strategy with only last state? 

        # TODO current strategy randomly samples index from the her_index till last 
        future_offset = np.random.uniform(size=batch_size) * (T - t_samples)
        future_offset = future_offset.astype(int)

        
        future_t = (t_samples + 1 + future_offset)[her_indexes]
        # replace go with achieved goal
        future_ag_image = episode_batch['achieved_goal_image'][episode_idxs[her_indexes], future_t]
        transitions['desired_goal_image'][her_indexes] = future_ag_image
        # to get the params to re-compute reward 
        # image based dense reward compute 
        transitions['reward'] = np.expand_dims(self.reward_func(transitions['achieved_goal_image_next'], transitions['desired_goal_image'],image_based=True), 1)
        transitions = {k: transitions[k].reshape(batch_size, *transitions[k].shape[1:]) for k in transitions.keys()}

        return transitions





In [12]:
(1,) + env.observation_space['image_observation'].shape 

(1, 3, 84, 84)

In [ ]:
class customherreplay():
    def __init__(self,observation_space, action_space,max_episode_stored, max_episode_length,goal_selection_strategy="future"):
        self.goal_selection_strategy = goal_selection_strategy
        action_dim = action_space.shape
        self.obs_shape = observation_space['image_observation'].shape
        self.goal_shape = observation_space['desired_goal_image'].shape
        self.max_episode_length = max_episode_length
        self.max_episode_stored = max_episode_stored
        input_shape = {
                "observation": (1,) + self.obs_shape,
                "achieved_goal": (1,) + self.goal_shape,
                "desired_goal": (1,) + self.goal_shape,
                "action": (action_dim,),
                "reward": (1,),
                "next_obs": (1,) + self.obs_shape,
                "next_achieved_goal": (1,) + self.goal_shape,
                "next_desired_goal": (1,) + self.goal_shape,
                "done": (1,),
            }
            self._observation_keys = ["observation", "achieved_goal", "desired_goal"]
            self._buffer = {
                key: np.zeros((self.max_episode_stored, self.max_episode_length, *dim), dtype=np.float32)
                for key, dim in input_shape.items()
            }
            # Store info dicts are it can be used to compute the reward (e.g. continuity cost)
            self.info_buffer = [deque(maxlen=self.max_episode_length) for _ in range(self.max_episode_stored)]
            # episode length storage, needed for episodes which has less steps than the maximum length
            self.episode_lengths = np.zeros(self.max_episode_stored, dtype=np.int64)
            
            
            
    def sample_goals(
        self,
        episode_indices: np.ndarray,
        her_indices: np.ndarray,
        transitions_indices: np.ndarray,
    ) -> np.ndarray:
        """
        Sample goals based on goal_selection_strategy.
        This is a vectorized (fast) version.
        :param episode_indices: Episode indices to use.
        :param her_indices: HER indices.
        :param transitions_indices: Transition indices to use.
        :return: Return sampled goals.
        """
        her_episode_indices = episode_indices[her_indices]

        if self.goal_selection_strategy == GoalSelectionStrategy.FINAL:
            # replay with final state of current episode
            transitions_indices = self.episode_lengths[her_episode_indices] - 1

        elif self.goal_selection_strategy == GoalSelectionStrategy.FUTURE:
            # replay with random state which comes from the same episode and was observed after current transition
            transitions_indices = np.random.randint(
                transitions_indices[her_indices] + 1, self.episode_lengths[her_episode_indices]
            )

        elif self.goal_selection_strategy == GoalSelectionStrategy.EPISODE:
            # replay with random state which comes from the same episode as current transition
            transitions_indices = np.random.randint(self.episode_lengths[her_episode_indices])

        else:
            raise ValueError(f"Strategy {self.goal_selection_strategy} for sampling goals not supported!")

        return self._buffer["achieved_goal"][her_episode_indices, transitions_indices]
    
    
    
    def _sample_transitions(
        self,
        batch_size: Optional[int],
        maybe_vec_env: Optional[VecNormalize],
        online_sampling: bool,
        n_sampled_goal: Optional[int] = None,
    ) -> Union[DictReplayBufferSamples, Tuple[Dict[str, np.ndarray], Dict[str, np.ndarray], np.ndarray, np.ndarray]]:
        """
        :param batch_size: Number of element to sample (only used for online sampling)
        :param env: associated gym VecEnv to normalize the observations/rewards
            Only valid when using online sampling
        :param online_sampling: Using online_sampling for HER or not.
        :param n_sampled_goal: Number of sampled goals for replay. (offline sampling)
        :return: Samples.
        """
        # Select which episodes to use
       
        assert batch_size is not None, "No batch_size specified for online sampling of HER transitions"
        # Do not sample the episode with index `self.pos` as the episode is invalid
        if self.full:
            episode_indices = (
                np.random.randint(1, self.n_episodes_stored, batch_size) + self.pos
            ) % self.n_episodes_stored
        else:
            episode_indices = np.random.randint(0, self.n_episodes_stored, batch_size)
        # A subset of the transitions will be relabeled using HER algorithm
        her_indices = np.arange(batch_size)[: int(self.her_ratio * batch_size)]
        
        ep_lengths = self.episode_lengths[episode_indices]

        # Special case when using the "future" goal sampling strategy
        # we cannot sample all transitions, we have to remove the last timestep
        if self.goal_selection_strategy == GoalSelectionStrategy.FUTURE:
            # restrict the sampling domain when ep_lengths > 1
            # otherwise filter out the indices
            her_indices = her_indices[ep_lengths[her_indices] > 1]
            ep_lengths[her_indices] -= 1

       
        # Select which transitions to use
        transitions_indices = np.random.randint(ep_lengths)
      
        # get selected transitions
        transitions = {key: self._buffer[key][episode_indices, transitions_indices].copy() for key in self._buffer.keys()}

        # sample new desired goals and relabel the transitions
        new_goals = self.sample_goals(episode_indices, her_indices, transitions_indices)
        transitions["desired_goal"][her_indices] = new_goals

        # Convert info buffer to numpy array
        transitions["info"] = np.array(
            [
                self.info_buffer[episode_idx][transition_idx]
                for episode_idx, transition_idx in zip(episode_indices, transitions_indices)
            ]
        )

        # Edge case: episode of one timesteps with the future strategy
        # no virtual transition can be created
        if len(her_indices) > 0:
            # Vectorized computation of the new reward
            transitions["reward"][her_indices, 0] = self.reward_func(
               
                # the new state depends on the previous state and action
                # s_{t+1} = f(s_t, a_t)
                # so the next_achieved_goal depends also on the previous state and action
                # because we are in a GoalEnv:
                # r_t = reward(s_t, a_t) = reward(next_achieved_goal, desired_goal)
                # therefore we have to use "next_achieved_goal" and not "achieved_goal"
                transitions["next_achieved_goal"][her_indices, 0],
                # here we use the new desired goal
                transitions["desired_goal"][her_indices, 0],
               
            )

        # concatenate observation with (desired) goal
        observations = self._normalize_obs(transitions, maybe_vec_env)

        # HACK to make normalize obs and `add()` work with the next observation
        next_observations = {
            "observation": transitions["next_obs"],
            "achieved_goal": transitions["next_achieved_goal"],
            # The desired goal for the next observation must be the same as the previous one
            "desired_goal": transitions["desired_goal"],
        }
        next_observations = self._normalize_obs(next_observations, maybe_vec_env)

        if online_sampling:
            next_obs = {key: self.to_torch(next_observations[key][:, 0, :]) for key in self._observation_keys}

            normalized_obs = {key: self.to_torch(observations[key][:, 0, :]) for key in self._observation_keys}

            return [observations=normalized_obs,
                actions=self.to_torch(transitions["action"]),
                next_observations=next_obs,
                dones=self.to_torch(transitions["done"]),
                rewards=self.to_torch((transitions["reward"]))]
                
            
        else:
            return observations, next_observations, transitions["action"], transitions["reward"]

In [13]:
from enum import Enum


class GoalSelectionStrategy(Enum):
    """
    The strategies for selecting new goals when
    creating artificial transitions.
    """

    # Select a goal that was achieved
    # after the current step, in the same episode
    FUTURE = 0
    # Select the goal that was achieved
    # at the end of the episode
    FINAL = 1
    # Select a goal that was achieved in the episode
    EPISODE = 2


# For convenience
# that way, we can use string to select a strategy
KEY_TO_GOAL_STRATEGY = {
    "future": GoalSelectionStrategy.FUTURE,
    "final": GoalSelectionStrategy.FINAL,
    "episode": GoalSelectionStrategy.EPISODE,
}